In [1]:
import tensorflow as tf
import numpy as np
import csv
import functools
import pandas as pd

In [2]:
test_data_path = "testdata_20.csv"
train_data_path = "traindata_80.csv"

In [3]:
LABEL_COLUMN = 'move'
LABELS = [0, 1, 2, 3]

In [4]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size = 5,
        label_name = LABEL_COLUMN,
        num_epochs = 1,
        ignore_errors = True,
        **kwargs)
    return dataset

In [10]:
raw_train_data = get_dataset(train_data_path)
raw_test_data = get_dataset(test_data_path)

In [12]:
tf.get_tensors()

AttributeError: module 'tensorflow' has no attribute 'get_tensors'

In [6]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [14]:
show_batch(raw_test_data)

1                   : [0.06666667 0.06666667 0.06666667 0.8666667  0.33333334]
2                   : [0.13333334 0.46666667 1.         0.93333334 0.46666667]
3                   : [0.2        0.73333335 0.4        0.33333334 0.53333336]
4                   : [0.26666668 0.2        0.93333334 0.         0.73333335]
5                   : [0.33333334 1.         0.46666667 0.73333335 0.6       ]
6                   : [0.4        0.         0.53333336 0.8        0.2       ]
7                   : [0.46666667 0.8666667  0.         0.26666668 0.26666668]
8                   : [0.53333336 0.6        0.8        0.13333334 0.6666667 ]
9                   : [0.6       0.6666667 0.2       0.4       0.4      ]
10                  : [0.6666667  0.8        0.33333334 1.         0.13333334]
11                  : [0.73333335 0.4        0.8666667  0.06666667 0.        ]
12                  : [0.8        0.33333334 0.13333334 0.53333336 0.06666667]
13                  : [0.8666667  0.93333334 0.6        0

In [18]:
CSV_COLUMNS=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','0','move']
temp_dataset = get_dataset(train_data_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

1                   : [0.33333334 0.26666668 0.33333334 0.6666667  0.06666667]
2                   : [0.06666667 0.6666667  0.06666667 0.26666668 0.2       ]
3                   : [0.2        0.46666667 0.13333334 0.53333336 0.26666668]
4                   : [0.26666668 0.4        0.2        0.33333334 0.53333336]
5                   : [0.6        0.06666667 0.6        0.13333334 0.33333334]
6                   : [0.13333334 0.8        0.46666667 0.2        0.46666667]
7                   : [0.46666667 0.8666667  0.26666668 0.93333334 0.8       ]
8                   : [0.53333336 0.13333334 0.8        0.06666667 1.        ]
9                   : [0.4        0.33333334 0.4        0.         0.6       ]
10                  : [0.6666667  0.6        0.73333335 0.4        0.13333334]
11                  : [0.         1.         0.53333336 0.73333335 0.73333335]
12                  : [0.73333335 0.53333336 0.         0.8        0.        ]
13                  : [0.8666667  0.93333334 0.86666

In [19]:
example_batch, labels_batch = next(iter(temp_dataset))

In [20]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [21]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[0.06666667 0.2        0.26666668 0.46666667 0.8666667  0.33333334
  0.13333334 0.53333336 0.6666667  0.         0.4        1.
  0.73333335 0.6        0.93333334 0.8       ]
 [0.06666667 0.13333334 0.2        0.26666668 0.4        0.
  0.46666667 0.53333336 0.33333334 0.6        0.6666667  0.73333335
  0.8666667  0.93333334 1.         0.8       ]
 [0.73333335 0.         0.8        0.26666668 0.4        0.06666667
  0.6666667  0.6        0.93333334 0.53333336 0.8666667  0.33333334
  0.2        0.46666667 1.         0.13333334]
 [0.13333334 0.73333335 0.33333334 0.46666667 0.06666667 0.8
  0.26666668 0.53333336 0.         0.2        0.4        1.
  0.6        0.8666667  0.6666667  0.93333334]
 [0.6        0.33333334 0.8        0.73333335 0.4        0.53333336
  0.26666668 0.         0.46666667 0.13333334 0.2        0.93333334
  1.         0.06666667 0.6666667  0.8666667 ]]

[1 1 0 0 2]


In [23]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [24]:
NUMERIC_FEATURES = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','0']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [25]:
show_batch(packed_train_data)

numeric             : [[0.33333334 0.6666667  0.06666667 0.6        0.73333335 0.
  0.93333334 0.13333334 0.46666667 0.4        0.53333336 0.8
  1.         0.2        0.26666668 0.8666667 ]
 [0.06666667 0.13333334 0.2        0.26666668 0.33333334 0.
  0.4        0.46666667 0.6        0.6666667  0.73333335 0.53333336
  0.8666667  0.93333334 1.         0.8       ]
 [0.46666667 0.2        0.06666667 0.33333334 0.13333334 0.53333336
  0.6666667  0.         0.6        0.4        0.8        0.26666668
  0.8666667  0.93333334 0.73333335 1.        ]
 [0.4        0.13333334 0.26666668 0.73333335 0.06666667 0.33333334
  0.6666667  0.2        0.6        1.         0.8666667  0.46666667
  0.93333334 0.         0.53333336 0.8       ]
 [0.06666667 0.4        0.53333336 0.13333334 0.         0.2
  0.46666667 0.26666668 0.33333334 0.93333334 0.6666667  0.8
  0.6        0.8666667  0.73333335 1.        ]]


In [26]:
example_batch, labels_batch = next(iter(packed_train_data))

In [27]:
desc = pd.read_csv(train_data_path)[NUMERIC_FEATURES].describe()
desc

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0
count,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000
mean,0.285116,0.320597,0.353121,0.374126,0.406388,0.423464,0.432982,0.470481,0.528790,0.532213,0.544110,0.570290,0.684125,0.658673,0.681648,0.733876
std,0.268076,0.265926,0.256098,0.231942,0.276902,0.283155,0.277411,0.254121,0.273233,0.302539,0.301554,0.281705,0.261505,0.297720,0.299037,0.276821
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.066667,0.133333,0.200000,0.266667,0.200000,0.200000,0.200000,0.266667,0.333333,0.333333,0.333333,0.400000,0.600000,0.466667,0.466667,0.600000
50%,0.133333,0.200000,0.266667,0.266667,0.333333,0.400000,0.466667,0.533333,0.600000,0.600000,0.600000,0.600000,0.866667,0.733333,0.733333,0.800000
75%,0.466667,0.466667,0.533333,0.533333,0.600000,0.666667,0.600000,0.600000,0.666667,0.733333,0.733333,0.800000,0.866667,0.933333,0.933333,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
numeric_column = tf.feature_column.numeric_column('numeric', shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(16,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [29]:
example_batch['numeric']

<tf.Tensor: id=1189, shape=(5, 16), dtype=float32, numpy=
array([[0.        , 0.26666668, 0.53333336, 1.        , 0.2       ,
        0.06666667, 0.46666667, 0.8666667 , 0.4       , 0.13333334,
        0.33333334, 0.93333334, 0.8       , 0.6666667 , 0.6       ,
        0.73333335],
       [0.06666667, 0.13333334, 0.2       , 0.26666668, 0.6       ,
        0.33333334, 0.46666667, 0.53333336, 0.4       , 0.93333334,
        0.        , 0.8       , 0.8666667 , 1.        , 0.73333335,
        0.6666667 ],
       [0.33333334, 0.06666667, 0.13333334, 0.26666668, 0.6       ,
        0.2       , 0.46666667, 0.53333336, 0.        , 0.8666667 ,
        1.        , 0.73333335, 0.93333334, 0.4       , 0.6666667 ,
        0.8       ],
       [0.4       , 0.8666667 , 0.06666667, 0.6       , 0.93333334,
        0.53333336, 0.46666667, 0.73333335, 0.33333334, 0.2       ,
        0.8       , 0.13333334, 0.6666667 , 0.        , 1.        ,
        0.26666668],
       [0.53333336, 0.4       , 0.26666668

In [30]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[0.        , 0.26666668, 0.53333336, 1.        , 0.2       ,
        0.06666667, 0.46666667, 0.8666667 , 0.4       , 0.13333334,
        0.33333334, 0.93333334, 0.8       , 0.6666667 , 0.6       ,
        0.73333335],
       [0.06666667, 0.13333334, 0.2       , 0.26666668, 0.6       ,
        0.33333334, 0.46666667, 0.53333336, 0.4       , 0.93333334,
        0.        , 0.8       , 0.8666667 , 1.        , 0.73333335,
        0.6666667 ],
       [0.33333334, 0.06666667, 0.13333334, 0.26666668, 0.6       ,
        0.2       , 0.46666667, 0.53333336, 0.        , 0.8666667 ,
        1.        , 0.73333335, 0.93333334, 0.4       , 0.6666667 ,
        0.8       ],
       [0.4       , 0.8666667 , 0.06666667, 0.6       , 0.93333334,
        0.53333336, 0.46666667, 0.73333335, 0.33333334, 0.2       ,
        0.8       , 0.13333334, 0.6666667 , 0.        , 1.        ,
        0.26666668],
       [0.53333336, 0.4       , 0.26666668, 0.2       , 0.        ,
        0.06666667, 0.6666667 , 

In [108]:
model = tf.keras.Sequential([
    numeric_layer,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(
loss = 'sparse_categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [32]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [117]:
model.fit(train_data, epochs=200)

Epoch 1/200
3000/3000 [==============================] - 9s 3ms/step - loss: 0.3813 - accuracy: 0.8574
Epoch 2/200
3000/3000 [==============================] - 6s 2ms/step - loss: 0.3664 - accuracy: 0.8630
Epoch 3/200
3000/3000 [==============================] - 8s 3ms/step - loss: 0.3803 - accuracy: 0.8588
Epoch 4/200
3000/3000 [==============================] - 9s 3ms/step - loss: 0.3657 - accuracy: 0.8666
Epoch 5/200
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3729 - accuracy: 0.8654
Epoch 6/200
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3762 - accuracy: 0.8648
Epoch 7/200
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3679 - accuracy: 0.8625
Epoch 8/200
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3653 - accuracy: 0.8631
Epoch 9/200
3000/3000 [==============================] - 6s 2ms/step - loss: 0.3775 - accuracy: 0.8615
Epoch 10/200
3000/3000 [==============================] - 6s 2ms/step - l

3000/3000 [==============================] - 5s 2ms/step - loss: 0.3647 - accuracy: 0.8650
Epoch 80/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3584 - accuracy: 0.8654
Epoch 81/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3715 - accuracy: 0.8637
Epoch 82/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3680 - accuracy: 0.8609
Epoch 83/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3650 - accuracy: 0.8690
Epoch 84/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3738 - accuracy: 0.8602
Epoch 85/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3752 - accuracy: 0.8636
Epoch 86/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3516 - accuracy: 0.8701
Epoch 87/200
3000/3000 [==============================] - 7s 2ms/step - loss: 0.3737 - accuracy: 0.8632
Epoch 88/200
3000/3000 [==============================] - 7s 2ms/step - loss:

3000/3000 [==============================] - 5s 2ms/step - loss: 0.3659 - accuracy: 0.8647
Epoch 158/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3720 - accuracy: 0.8642
Epoch 159/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3918 - accuracy: 0.8588
Epoch 160/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3669 - accuracy: 0.8624
Epoch 161/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3634 - accuracy: 0.8665
Epoch 162/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3640 - accuracy: 0.8670
Epoch 163/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3666 - accuracy: 0.8639
Epoch 164/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3686 - accuracy: 0.8648
Epoch 165/200
3000/3000 [==============================] - 5s 2ms/step - loss: 0.3751 - accuracy: 0.8647
Epoch 166/200
3000/3000 [==============================] - 5s 2ms/ste

In [118]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

758/758 [==============================] - 2s 2ms/step - loss: 0.8600 - accuracy: 0.7365


Test Loss 0.8600283456972578, Test Accuracy 0.73653644323349


In [114]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

758/758 [==============================] - 2s 2ms/step - loss: 0.8298 - accuracy: 0.7368


Test Loss 0.8297876721303186, Test Accuracy 0.7368004322052002
